In [41]:
from taxcalc import *
from bokeh.io import show, output_notebook
from bokeh.charts import Bar, Scatter
from bokeh.layouts import column
from collections import OrderedDict
import copy
from notebookfunctions import distribution, index_list
output_notebook()

Loading BokehJS ...

### Missing Variables

In [42]:
cps = pd.read_csv('../taxdata/cps_data/cps.csv')
usable = open('taxcalc/records_variables.json')
usable_vars = json.load(usable)
usable.close()
missing = 0
for item in usable_vars['read'].keys():
    if item not in cps.columns:
        print '{}: {}'.format(item, usable_vars['read'][item]['desc'])
        missing += 1
print '\nTotal Variables Missing: {}'.format(missing)

p23250: Sch D: Net long-term capital gains/losses
p25470: Sch E: Royalty depletion and/or rental depreciation
e09800: Unreported payroll taxes from Form 4137 or 8919
e02000: Sch E rental, royalty, S-corp, etc, income/loss
e62900: Alternative Minimum Tax foreign tax credit from Form 6251
p08000: Other tax credits (but not including Sch R credit)
e58990: Investment income elected amount from Form 4952
e00700: Taxable refunds of state and local income taxes
e03290: Health savings account deduction from Form 8889
e07240: Retirement savings contributions credit from Form 8880
agi_bin: Historical AGI category used in data extrapolation
e27200: Sch E: Farm rent net income or loss
e01200: Other net gain/loss from Form 4797
e03500: Alimony paid
e07260: Residential energy credit from Form 5695
p22250: Sch D: Net short-term capital gains/losses
e03220: Educator expenses
e07400: General business credit from Form 3800
f6251: 1 if Form 6251 (AMT) attached to return; otherwise 0
e03230: Tuition and f

In [45]:
# Create calculator from the PUF
calc_puf = Calculator(records=Records(), policy=Policy())
calc_puf.advance_to_year(2017)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [46]:
# Calculator the CPS
wt = pd.read_csv('../Dropbox/cps_weights.csv')
# wt.drop('SEQUENCE', inplace=True, axis=1)
# wt = wt.loc[:,:'WT2026']
wt *= 100
recs_cps = Records(data=cps,
                   weights=wt,
                   adjust_ratios=None,
                   start_year=2015)
pol_cps = Policy(start_year=2015, num_years=11)
calc_cps = Calculator(records=recs_cps, policy=pol_cps)
calc_cps.advance_to_year(2017)

In [47]:
calc_puf.calc_all()
calc_cps.calc_all()

### CPS Distribution Table

In [48]:
utils.create_distribution_table(calc_cps.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"1,089","16,762,987",723,"9,476,876","6,279","14,884",6,"5,022",23,1,720,0,0,1,0,0,238,-237,166,-71
1,"9,131","16,762,619","4,891","11,505,449","7,567","201,891",132,"5,820",125,9,"4,830","11,189",0,9,2,0,"2,030","-2,023",678,"-1,345"
2,"16,266","16,763,717","9,244","12,012,549","7,708","1,092,198",748,"6,285","1,621",154,"8,834","28,258",1,155,28,0,"3,530","-3,404","1,286","-2,117"
3,"24,244","16,763,003","17,424","12,058,174","7,232","3,421,260","2,152","7,006","5,382",570,"16,110","34,697",2,572,97,0,"4,203","-3,728","2,435","-1,294"
4,"33,864","16,763,511","27,153","11,055,477","6,627","5,314,344","3,708","7,344","11,736","1,374","24,829","29,524",4,"1,377",178,0,"3,284","-2,085","3,723","1,637"
5,"46,160","16,762,996","39,586","9,369,394","5,764","7,319,393","6,128","7,825","20,865","2,550","35,733","31,850",5,"2,555",294,0,"1,481",780,"5,237","6,017"
6,"61,998","16,763,304","55,811","7,486,171","4,826","9,265,319","9,189","8,289","33,899","4,551","50,104","28,143",4,"4,555",399,0,163,"3,993","6,969","10,962"
7,"84,003","16,763,079","77,540","5,288,866","3,576","11,470,220","13,200","9,033","51,957","7,621","69,650","35,128",6,"7,627",481,0,13,"7,133","9,464","16,597"
8,"118,863","16,761,269","110,382","3,226,875","2,302","13,532,226","18,357","10,027","79,808","12,515","99,812","38,759",5,"12,520",469,0,2,"12,049","13,400","25,449"
9,"265,852","16,765,315","251,802","1,362,512","1,014","15,402,146","29,067","9,634","212,159","48,026","236,808","3,308,641",753,"48,778",48,309,0,"49,039","21,906","70,944"


### PUF Distribution Table

In [49]:
utils.create_distribution_table(calc_puf.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"-6,694","17,041,816","-6,948","11,516,212","6,294","4,682",21,"4,443",58,7,"-7,360",0,0,7,0,7,137,-124,340,216
1,"8,807","17,040,415","6,002","14,010,959","7,370","53,927",27,"4,948",315,28,"5,974","14,445",0,28,1,5,743,-710,844,134
2,"15,943","17,041,505","10,265","13,678,936","8,082","312,986",210,"6,559","1,631",160,"10,122","16,510",1,161,20,7,"1,634","-1,486","1,469",-17
3,"23,438","17,044,022","15,739","14,832,468","8,336","731,409",590,"6,978","4,341",463,"15,373","35,460",2,465,93,13,"1,414","-1,028","2,105","1,077"
4,"32,816","17,041,666","24,306","14,799,151","8,250","1,706,190","1,483","7,292","10,012","1,162","23,372","19,038",2,"1,164",255,18,974,-47,"3,080","3,034"
5,"44,317","17,042,127","35,697","13,772,526","7,913","3,146,508","2,982","7,429","18,933","2,333","33,909","25,706",4,"2,337",405,30,419,"1,544","4,233","5,776"
6,"59,254","17,040,903","50,971","11,779,967","7,175","5,241,174","5,537","7,768","30,964","4,164","47,677","25,635",5,"4,169",511,45,122,"3,580","5,685","9,266"
7,"81,104","17,042,185","74,150","9,416,430","6,147","7,617,314","8,813","8,592","50,717","7,482","69,308","72,693",8,"7,490",655,68,77,"6,826","8,400","15,226"
8,"120,974","17,042,904","112,646","5,994,841","4,253","11,044,938","15,074","10,184","83,182","13,455","104,600","253,699",32,"13,487",796,105,95,"12,700","13,770","26,470"
9,"367,045","17,041,967","348,501","2,092,487","1,497","14,937,255","39,706","9,557","298,179","75,025","332,039","5,108,152","2,582","77,607","1,643","2,566",47,"78,483","24,237","102,721"


### Diagnostic Table Comparison

In [50]:
cps_diag = utils.create_diagnostic_table(calc_cps)

In [51]:
puf_diag = utils.create_diagnostic_table(calc_puf)

In [52]:
diag_data = pd.DataFrame()
diag_data['CPS'] = cps_diag[2017]
diag_data['PUF'] = puf_diag[2017]
diag_data['% Change'] = ((cps_diag[2017] / puf_diag[2017]) - 1) * 100

In [53]:
diag_data

,CPS,PUF,% Change
Returns (#m),167.6,170.4,-1.6
AGI ($b),"9,967.0","11,440.8",-12.9
Itemizers (#m),67.0,44.8,49.6
Itemized Deduction ($b),"1,386.1","1,268.7",9.3
Standard Deduction Filers (#m),82.8,111.9,-26.0
Standard Deduction ($b),736.3,996.2,-26.1
Personal Exemption ($b),"1,168.1","1,175.7",-0.6
Taxable Income ($b),"7,000.2","8,492.6",-17.6
Regular Tax ($b),"1,297.0","1,777.1",-27.0
AMT Income ($b),"9,177.0","10,822.0",-15.2


In [54]:
# Total value of missing itemized deductions
in_billions = 1e-9
state = (calc_puf.records.e18400 * calc_puf.records.s006).sum()
print 'State and Local: {} ($b)'.format(round(state * in_billions, 2))
int_paid = (calc_puf.records.e19200 * calc_puf.records.s006).sum()
print 'Interest Paid: {} ($b)'.format(round(int_paid * in_billions, 2))
net_cas = (calc_puf.records.e20500 * calc_puf.records.s006).sum()
print 'Net Casualty or Theft Loss: {} ($b)'.format(round(net_cas * in_billions, 2))
print '-----------------'
print 'Total: {} ($b)'.format((round((state + int_paid + net_cas) * in_billions, 2)))

State and Local: 364.93 ($b)
Interest Paid: 379.23 ($b)
Net Casualty or Theft Loss: 3.53 ($b)
-----------------
Total: 747.69 ($b)


### Income Levels

In [55]:
inc_dict = OrderedDict()
inc_dict['CPS'] = [] 
inc_dict['PUF'] = []
inc_dict['Diff'] = []
inc_dict['Pct Diff'] = []
inc_list = ['WAS', 'Taxable Interest', 'Ordinary Dividends', 'Qualified Dividends', 'Business Income']
was_cps = (calc_cps.records.e00200 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(was_cps)
was_puf = (calc_puf.records.e00200 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(was_puf)
was_diff = (was_cps - was_puf)
inc_dict['Diff'].append(was_diff)
inc_dict['Pct Diff'].append((was_diff / was_puf) * 100)

int_cps = (calc_cps.records.e00300 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(int_cps)
int_puf = (calc_puf.records.e00300 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(int_puf)
int_diff = (int_cps - int_puf)
inc_dict['Diff'].append(int_diff)
inc_dict['Pct Diff'].append((int_diff / int_puf) * 100)

odiv_cps = (calc_cps.records.e00600 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(odiv_cps)
odiv_puf = (calc_puf.records.e00600 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(odiv_puf)
odiv_diff = (odiv_cps - odiv_puf)
inc_dict['Diff'].append(odiv_diff)
inc_dict['Pct Diff'].append((odiv_diff / odiv_puf) * 100)

qdiv_cps = (calc_cps.records.e00650 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(qdiv_cps)
qdiv_puf = (calc_puf.records.e00650 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(qdiv_puf)
qdiv_diff = (qdiv_cps - qdiv_puf)
inc_dict['Diff'].append(qdiv_diff)
inc_dict['Pct Diff'].append((qdiv_diff / qdiv_puf) * 100)

biz_cps = (calc_cps.records.e00900 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(biz_cps)
biz_puf = (calc_puf.records.e00900 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(biz_puf)
biz_diff = (biz_cps - biz_puf)
inc_dict['Diff'].append(biz_diff)
inc_dict['Pct Diff'].append((biz_diff / biz_puf) * 100)

inc_df = pd.DataFrame.from_dict(inc_dict)
inc_df.index = inc_list
inc_df

,CPS,PUF,Diff,Pct Diff
WAS,"7,344,071,812,573.8","7,810,139,100,122.9","-466,067,287,549.0",-6.0
Taxable Interest,"105,243,551,916.6","105,440,323,156.2","-196,771,239.6",-0.2
Ordinary Dividends,"267,521,441,047.3","304,756,093,045.9","-37,234,651,998.6",-12.2
Qualified Dividends,"202,139,200,855.4","222,057,250,910.0","-19,918,050,054.6",-9.0
Business Income,"331,254,028,372.3","352,064,769,480.1","-20,810,741,107.8",-5.9


### Distribution of Income Variables

In [56]:
# Generate data for distribution plots
cps_dist = pd.DataFrame()
puf_dist = pd.DataFrame()
cps_was = distribution(calc_cps.records.e00200, calc_cps.records.s006, calc_cps.records.c00100)
puf_was = distribution(calc_puf.records.e00200, calc_puf.records.s006, calc_puf.records.c00100)
cps_int = distribution(calc_cps.records.e00300, calc_cps.records.s006, calc_cps.records.c00100)
puf_int = distribution(calc_puf.records.e00300, calc_puf.records.s006, calc_puf.records.c00100)
cps_odiv = distribution(calc_cps.records.e00600, calc_cps.records.s006, calc_cps.records.c00100)
puf_odiv = distribution(calc_puf.records.e00600, calc_puf.records.s006, calc_puf.records.c00100)
cps_qdiv = distribution(calc_cps.records.e00650, calc_cps.records.s006, calc_cps.records.c00100)
puf_qdiv = distribution(calc_puf.records.e00650, calc_puf.records.s006, calc_puf.records.c00100)
cps_biz = distribution(calc_cps.records.e00900, calc_cps.records.s006, calc_cps.records.c00100)
puf_biz = distribution(calc_puf.records.e00900, calc_puf.records.s006, calc_puf.records.c00100)
cps_dist['WAS'] = cps_was[1]
puf_dist['WAS'] = puf_was[1]
cps_dist['INT'] = cps_int[1]
puf_dist['INT'] = puf_int[1]
cps_dist['ODIV'] = cps_odiv[1]
puf_dist['ODIV'] = puf_odiv[1]
cps_dist['QDIV'] = cps_qdiv[1]
puf_dist['QDIV'] = puf_qdiv[1]
cps_dist['BIZ'] = cps_biz[1]
puf_dist['BIZ'] = puf_biz[1]
cps_dist['AGI Bin'] = index_list()
puf_dist['AGI Bin'] = index_list()
cps_dist['label'] = 'CPS'
puf_dist['label'] = 'PUF'
scatter_data = pd.concat([cps_dist, puf_dist])
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
scatter_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Percent of Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(scatter_data, x='AGI Bin', y=item[0], color='label', ylabel='Percent',
                      title=title, tooltips=[('PCT', '@{}'.format(item[0]))])
    scatter_list.append(scatter)

In [57]:
show(column(scatter_list))

### Itemized Deduction Amounts

In [58]:
deductions_cps = {'Medical Expenses': (calc_cps.records.e17500 * calc_cps.records.s006).sum(),
                  'State and Local Taxes':  (calc_cps.records.e18400 * calc_cps.records.s006).sum(),
                  'Real Estate Taxes':  (calc_cps.records.e18500 * calc_cps.records.s006).sum(),
                  'Interest Paid':  (calc_cps.records.e19200 * calc_cps.records.s006).sum(),
                  'Charitable Cash Contributions': (calc_cps.records.e19800 * calc_cps.records.s006).sum(),
                  'Charitable Non-Cash Contributions': (calc_cps.records.e20100 * calc_cps.records.s006).sum(),
                  'Total Misc. Expenses':  (calc_cps.records.e20400* calc_cps.records.s006).sum(),
                  'Net Casualty or Loss': (calc_cps.records.e20500 * calc_cps.records.s006).sum()}
ded_cps_df = pd.DataFrame.from_dict(deductions_cps, 'index')
ded_cps_df.columns = ['Total']
ded_cps_df['source'] = 'CPS'

deductions_puf = {'Medical Expenses': (calc_puf.records.e17500 * calc_puf.records.s006).sum(),
                  'State and Local Taxes':  (calc_puf.records.e18400 * calc_puf.records.s006).sum(),
                  'Real Estate Taxes':  (calc_puf.records.e18500 * calc_puf.records.s006).sum(),
                  'Interest Paid':  (calc_puf.records.e19200 * calc_puf.records.s006).sum(),
                  'Charitable Cash Contributions': (calc_puf.records.e19800 * calc_puf.records.s006).sum(),
                  'Charitable Non-Cash Contributions': (calc_puf.records.e20100 * calc_puf.records.s006).sum(),
                  'Total Misc. Expenses':  (calc_puf.records.e20400* calc_puf.records.s006).sum(),
                  'Net Casualty or Loss': (calc_puf.records.e20500 * calc_puf.records.s006).sum()}
ded_puf_df = pd.DataFrame.from_dict(deductions_puf, 'index')
ded_puf_df.columns = ['Total']
ded_puf_df['source'] = 'PUF'

ded_full_df = pd.concat([ded_cps_df, ded_puf_df])
ded_full_df['index'] = ded_full_df.index

In [59]:
ded_bar = Bar(ded_full_df, 'index', 'Total', group='source', title='Itemized Deduction Totals',
              xlabel='Deduction', ylabel='Total', tooltips=[('Deduction', '@index'), ('Total', '@height'),
                                                            ('Data', '@source')])
show(ded_bar)

In [60]:
calc_cps.records.e03210.sum()

41484511.864319362